In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

import pyDOE

import pandas as pd
import numpy as np

import logging
import datetime

import matplotlib.pyplot as plt
from scipy.interpolate import griddata

In [2]:
init_dict = {
    "Glorot uniform": torch.nn.init.xavier_uniform_,
    "He uniform": torch.nn.init.kaiming_uniform_,
    "zeros": torch.nn.init.zeros_,
    }

class MLP(nn.Module):
    def __init__(self, n_input, n_hidden, n_output, n_layers=1, act=nn.GELU(), residual=False):
        super(MLP, self).__init__()
        self.n_input = n_input # input layer dim
        self.n_hidden = n_hidden # hidden layer dim
        self.n_output = n_output # output layer dim
        self.n_layers = n_layers # count of hidden layers
        self.act = act # activation function
        self.residual = residual
        
        # layers
        self.linear_pre = nn.Linear(n_input, n_hidden)
        self.linears_hidden = nn.ModuleList([nn.Linear(n_hidden, n_hidden) for _ in range(n_layers)])
        self.linear_post = nn.Linear(n_hidden, n_output)
        
    def init_weights(self):
        weight_init = "He uniform" if self.act == nn.ReLU() else "Glorot uniform"
        weight_init = init_dict.get(weight_init)
        bias_init = init_dict.get("zeros")
        gain = nn.init.calculate_gain('leaky_relu') if isinstance(self.act, nn.LeakyReLU) else \
               nn.init.calculate_gain('tanh') if isinstance(self.act, nn.Tanh) else \
               nn.init.calculate_gain('linear')
        
        # weights init
        weight_init(self.linear_pre.weight, gain=gain)
        weight_init(self.linear_post.weight, gain=gain)
        for linear in self.linears_hidden:
            weight_init(linear.weight, gain=gain)
        
        # bias init
        bias_init(self.linear_pre.bias)
        bias_init(self.linear_post.bias)
        for linear in self.linears_hidden:
            bias_init(linear.bias)

    def forward(self, x):
        x = self.act(self.linear_pre(x))
        for i in range(self.n_layers):
            x = x + self.act(self.linears_hidden[i](x)) if self.residual else self.act(self.linears_hidden[i](x))
        x = self.linear_post(x)
        return x

class DeepONet(nn.Module):
    def __init__(self, branch_info, truck_info, act=nn.GELU(), residual=False):
        super(DeepONet, self).__init__()
        self.branch_info = branch_info
        self.truck_info = truck_info
        self.act = act
        self.residual = residual
    
        # branch and truck network
        self.branch_net = MLP(
            branch_info["n_input"],
            branch_info["n_hidden"],
            branch_info["n_output"],
            branch_info["n_layers"],
            self.act, self.residual)
        self.truck_net = MLP(
            truck_info["n_input"],
            truck_info["n_hidden"],
            truck_info["n_output"],
            truck_info["n_layers"],
            self.act, self.residual)
        
    def merge_branch_trunk(self, x_func, x_loc):
        y = torch.einsum('bi,bi->b', x_func, x_loc)
        y = torch.unsqueeze(y, dim=1)
        return y
    
    def forward(self, x_func, x_loc):
        x_branch = self.branch_net(x_func)
        x_truck = self.truck_net(x_loc)
        y = self.merge_branch_trunk(x_branch, x_truck)
        return y

In [3]:
def poisson_equation(x,y,a=1,b=1,c=1):
    return a * torch.sin(x/b) * torch.sin(y/c)

def make_deeponet_dataset(lb, ub, branch_samples, trunk_samples, coef):
    domain_length = ub - lb
    a,b,c = coef
    # trunk_data 생성: 예측할 [x, y] 위치를 생성합니다.
    trunk_data = pyDOE.lhs(2, samples=trunk_samples)
    trunk_data = trunk_data * domain_length - domain_length/2

    # trunk_data 위치에서의 실제 u(x, y) 값 계산
    y_true = torch.tensor([poisson_equation(torch.tensor(x[0]), torch.tensor(x[1]), a=a,b=b,c=c).item() for x in trunk_data], dtype=torch.float32)
    trunk_data = torch.tensor(trunk_data, dtype=torch.float32)
    
    # branch_data 생성: 함수 값 샘플을 생성합니다. 여기서는 더 많은 포인트를 사용합니다.
    branch_points = pyDOE.lhs(2, samples=branch_samples)
    branch_points = branch_points * domain_length - domain_length/2
    branch_data = torch.tensor([poisson_equation(torch.tensor(x[0]), torch.tensor(x[1]), a=a,b=b,c=c).item() for x in branch_points], dtype=torch.float32)

    return trunk_data, branch_data, y_true[:, None]

# Sample with Poisson equation

## Problem setting

학습하고자 하는 PDE는 아래와 같다.

$$
\begin{split}
u_{xx}+u_{yy}=f & \ \text{in}\ \Omega \\
% u=0 & \ \text{on} \ \partial \Omega
\end{split}
$$

이때 Original u가

$$
u =  a\sin(\frac{x}{b})\sin(\frac{y}{c})
$$
일 때,

$u_{xx}$와 $u_{yy}$ 는 다음과 같다.

$$
\begin{split}
u_{xx} & = -\frac{a}{b^2} \sin\left(\frac{x}{b}\right) \sin\left(\frac{y}{c}\right) \\
u_{yy} & = -\frac{a}{c^2} \sin\left(\frac{x}{b}\right) \sin\left(\frac{y}{c}\right)
\end{split}
$$

즉

$$
\begin{split}
u_{xx}+u_{yy} &= -a \left(\frac{1}{b^2} + \frac{1}{c^2}\right) \sin\left(\frac{x}{b}\right) \sin\left(\frac{y}{c}\right)\\
&= -\left(\frac{1}{b^2} + \frac{1}{c^2}\right) a \sin\left(\frac{x}{b}\right) \sin\left(\frac{y}{c}\right) \\
&= -\left(\frac{1}{b^2} + \frac{1}{c^2}\right) u
\end{split}
$$

가 되어 PDE 문제가 된다.

따라서 PINN은 NN이 예측한 $f_\text{pred}$ 에 대하여 $f_{xx} + f_{yy} + \left(\frac{1}{b^2} + \frac{1}{c^2}\right) f = 0$ 에 대하여 Loss를 설정하여 학습한다.

In [4]:
# logging 설정
now = datetime.datetime.now()
log_filename = now.strftime("%Y%m%d%H%M%S") + "_training.log"
logging.basicConfig(filename=log_filename, level=logging.INFO, 
                    format='%(asctime)s:%(levelname)s:%(message)s')
logging.info("=============================== Start ===============================")

# 학습 device
print("Number of GPUs:", torch.cuda.device_count())
for i in range(torch.cuda.device_count()):
    print("GPU", i+1, ":", torch.cuda.get_device_name(i))
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}') # 1개의 gpu로 진행

# pde coefficient data
coef_df = pd.read_csv('NewABC_cluster_results_newsdist(0.1).csv').iloc[:,:3]

# hyperparameters
epochs = 100
batch_size = 100

lb = -2*torch.pi
ub = 2*torch.pi
branch_samples = 100
trunk_samples = 1000
coef = (3,1,1) # sample coefficient

branch_info = {
    "n_input": 100,
    "n_hidden": 64,
    "n_output": 32,
    "n_layers": 3,
    }
truck_info = {
    "n_input": 2,
    "n_hidden": 64,
    "n_output": 32,
    "n_layers": 3,
    }
act = nn.GELU()

# coef 반복
for idx, row in coef_df.iterrows():
    coef = (row["A"], row["B"], row["C"])
    logging.info(f"coef: {coef} training...")
    # 데이터셋 생성
    trunk_data, branch_data, y_true = make_deeponet_dataset(lb, ub, branch_samples, trunk_samples, coef)
    branch_data = branch_data.repeat(1000,1)
    dataset = TensorDataset(trunk_data, branch_data, y_true)
    data_loader = DataLoader(dataset, batch_size=50, shuffle=True, pin_memory=True, num_workers=2)

    # 모델 생성 및 컴파일
    deeponet = DeepONet(branch_info, truck_info, act=nn.GELU(), residual=False)
    # deeponet = torch.nn.DataParallel(deeponet) # multi-GPU 사용
    deeponet.to(device)  # 모델을 GPU로 이동
    
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(deeponet.parameters(), lr=0.001)

    # 학습
    for epoch in range(epochs):
        deeponet.train()
        total_loss = 0
        for trunk_input, branch_input, targets in data_loader:
            trunk_input = trunk_input.to(device)
            branch_input = branch_input.to(device)
            targets = targets.to(device)
            outputs = deeponet(branch_input, trunk_input)
            loss = criterion(outputs, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        logging.info(f'Epoch {epoch+1}, Average Loss: {total_loss / len(data_loader)}')
        print(f'Epoch {epoch+1}, Average Loss: {total_loss / len(data_loader)}')
    logging.info(f"coef: {coef} training done!")
    logging.info("---------------------------------------------------------------")
    
    # test 시각화
    deeponet.to('cpu')
    
    input_list = []
    output_list = []

    deeponet.eval()
    for trunk_inputs, branch_input, targets in data_loader:
        outputs = deeponet(branch_input, trunk_inputs)
        input_list.append(trunk_inputs)
        output_list.append(outputs)
        
    input_list = torch.cat(input_list, dim=0).detach().numpy()
    output_list = torch.cat(output_list, dim=0).detach().numpy()
    
    grid_x, grid_y = np.mgrid[-2*np.pi:2*np.pi:100j, -2*np.pi:2*np.pi:100j] 
    grid_z = griddata((input_list[:,0], input_list[:,1]), output_list.reshape(-1), (grid_x, grid_y), method='cubic')

    plt.figure(figsize=(8, 8))
    plt.imshow(grid_z.T, extent=(-2*np.pi,2*np.pi, -2*np.pi,2*np.pi), origin='lower', cmap='viridis')
    plt.colorbar(label=f'f(x, y) = {row["A"]}*sin(x/{row["B"]}) * cos(y/{row["C"]})')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Interpolated Visualization of LHS Sampling and Function')
    plt.savefig(f'{row["A"]}_{row["B"]}_{row["C"]}.jpg')
    plt.close()

Number of GPUs: 2
GPU 1 : NVIDIA TITAN Xp
GPU 2 : NVIDIA TITAN Xp
Using device: cuda
Epoch 1, Average Loss: 0.1759501237422228
Epoch 2, Average Loss: 0.06585810016840696
Epoch 3, Average Loss: 0.024955237470567226
Epoch 4, Average Loss: 0.013465915038250387
Epoch 5, Average Loss: 0.010304639604873956
Epoch 6, Average Loss: 0.00770927662961185
Epoch 7, Average Loss: 0.006216852075885982
Epoch 8, Average Loss: 0.005294714029878378
Epoch 9, Average Loss: 0.003897534601856023
Epoch 10, Average Loss: 0.0036875890800729394
Epoch 11, Average Loss: 0.004160078731365502
Epoch 12, Average Loss: 0.002064346871338785
Epoch 13, Average Loss: 0.0011961141572101042
Epoch 14, Average Loss: 0.0007033416230115108
Epoch 15, Average Loss: 0.0005845902662258596
Epoch 16, Average Loss: 0.0003601222197175957
Epoch 17, Average Loss: 0.00036926124012097715
Epoch 18, Average Loss: 0.00034923511921078896
Epoch 19, Average Loss: 0.00039982391826924867
Epoch 20, Average Loss: 0.00031600139118381776
Epoch 21, Avera